In [ ]:
import galois
import numpy as np
import sys
sys.path.append('../../doubling-CSST/')
from convert_alist import readAlist
alistDirPath = "../../doubling-CSST/alistMats/GO03_self_dual/"
# The following are the parameters of the self-dual code included in the alistDirPath
length_dist_dict = {4:2, 6:2, 8:2, 10:2, 12:4, 14:4, 16:4, 18:4, 20:4, 22:6, 24:6, 26:6, 28:6, 30:6, 32:8, 34:6, 36:8, 38:8, 40:8, 42:8, 44:8, 46:8, 48:8, 50:8, 52:10, 54:8, 56:10, 58:10, 60:12, 62:10, 64:10}

n = 12 # all even lengths from 4 to 64
d = length_dist_dict[n]
F2 = galois.GF(2)
alistFilePath = alistDirPath + "n" + str(n) + "_d" + str(d) + ".alist"

GenMat = F2(readAlist(alistFilePath))
G_punctured = GenMat[:, :-1]  # Puncture the last column
print("Punctured code: [n=%s,k=%s]" % (G_punctured.shape[1], G_punctured.shape[0]))
H_punctured = G_punctured.null_space() # Parity-check matrix of the punctured code = generator matrix of the dual of the punctured code
print("Dual of the punctured code: [n=%s,k=%s]" % (H_punctured.shape[1], H_punctured.shape[0]))
print(H_punctured)
H_CSS = np.block([
    [H_punctured, np.zeros_like(H_punctured)],
    [np.zeros_like(H_punctured), H_punctured]
]); 
print("The sympletic matrix of the self-dual CSS code [[n=%s,k=%s, d>=%s]]:\n" % (H_punctured.shape[1], H_punctured.shape[1]-2*H_punctured.shape[0], d-1))
H_CSS

Punctured code: [n=11,k=6]
Dual of the punctured code: [n=11,k=5]
[[1 0 1 0 0 0 0 0 1 0 1]
 [0 1 1 0 0 1 1 0 1 1 0]
 [0 0 0 1 0 1 0 0 1 0 1]
 [0 0 0 0 1 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 1 1 1 1]]
The sympletic matrix of the self-dual CSS code [[n=11,k=1, d>=3]]:



array([[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]],
      dtype=uint8)

In [52]:
import numpy as np
from mwpf import HyperEdge, SolverInitializer, SolverSerialJointSingleHair, SyndromePattern

# --- 1. Corrected Graph Conversion ---
def convert_to_mwpf_hyperedges_correct(matrix, weight):
    """
    Correct mapping for QEC Decoding:
    - Vertices = Rows of H (Checks)
    - Hyperedges = Columns of H (Qubits)
    """
    if hasattr(matrix, 'view'):
        mat_np = np.array(matrix, dtype=int)
    else:
        mat_np = matrix

    num_rows, num_cols = mat_np.shape
    edges_list = []

    # Iterate over Columns (Qubits)
    for c in range(num_cols):
        # Find which checks (rows) this qubit affects
        # We look at the column 'c' across all rows
        check_indices = np.nonzero(mat_np[:, c])[0].tolist()
        
        # If a qubit affects checks [0, 1], it is an edge between Node 0 and Node 1.
        if len(check_indices) > 0:
            edges_list.append(HyperEdge(check_indices, weight))
        else:
            # Handle qubits that don't touch any checks (if any) or preserve index alignment
            # Usually we just skip them or add an empty edge, 
            # but for alignment, it's safer to just append the edge even if empty 
            # (though strictly inactive).
            edges_list.append(HyperEdge([], weight))
        
    # The number of vertices in the solver is the number of CHECKS (rows)
    return num_rows, edges_list


In [ ]:
vertex_num, weighted_edges = convert_to_mwpf_hyperedges_correct(H_CSS, DEFAULT_WEIGHT)

print(f"Solver Vertices (Checks): {vertex_num}") 
print(f"Solver Edges (Qubits): {len(weighted_edges)}")

initializer = SolverInitializer(vertex_num, weighted_edges)
hyperion = SolverSerialJointSingleHair(initializer)

Solver Vertices (Checks): 10
Solver Edges (Qubits): 22


In [54]:
bias_factor = 0.0
num_shots = 10
error_rate = 0.1

rZ = bias_factor / (1 + bias_factor)
rX = rY = (1 - rZ) / 2

error_x = np.random.choice(
    [0, 1], size=(num_shots, H_punctured.shape[1]), p=[1 - rX * error_rate, rX * error_rate]
)
error_y = np.random.choice(
    [0, 1], size=(num_shots, H_punctured.shape[1]), p=[1 - rY * error_rate, rY * error_rate]
)
error_z = np.random.choice(
    [0, 1], size=(num_shots, H_punctured.shape[1]), p=[1 - rZ * error_rate, rZ * error_rate]
)

error_x = (error_x + error_y) % 2
error_z = (error_z + error_y) % 2

syndromes_x = (error_z @ np.array(H_punctured, dtype=int).T) % 2
syndromes_z = (error_x @ np.array(H_punctured, dtype=int).T) % 2

syndromes = np.hstack([syndromes_x, syndromes_z])

for i in range(num_shots):
    # Get indices of active syndromes (Checks that failed)
    syndrome_indices = np.nonzero(syndromes[i])[0].tolist()
    
    if not syndrome_indices:
        print(f"Shot {i}: Clean")
        continue

    # Solve
    hyperion.solve(SyndromePattern(syndrome_indices))
    
    # The subgraph returns the indices of the *Edges* (Qubits) that were matched
    hyperion_subgraph = hyperion.subgraph()
    print(hyperion_subgraph)  # out: [3, 5], weighted 160
    _, bound = hyperion.subgraph_range()
    print((bound.lower, bound.upper))  # out: (Fraction(160, 1), Fraction(160, 1))

[19]
(OrderedFloat(100.0), OrderedFloat(100.0))
[11, 21]
(OrderedFloat(200.0), OrderedFloat(200.0))
[19]
(OrderedFloat(100.0), OrderedFloat(100.0))
Shot 3: Clean
Shot 4: Clean
[14]
(OrderedFloat(100.0), OrderedFloat(100.0))
[21]
(OrderedFloat(100.0), OrderedFloat(100.0))
[12]
(OrderedFloat(100.0), OrderedFloat(100.0))
[1, 18]
(OrderedFloat(200.0), OrderedFloat(200.0))
[17]
(OrderedFloat(100.0), OrderedFloat(100.0))


In [56]:
import numpy as np
from mwpf import SyndromePattern

bias_factor = 0.0
num_shots = 10
error_rate = 0.1
DEFAULT_WEIGHT = 100

rZ = bias_factor / (1 + bias_factor)
rX = rY = (1 - rZ) / 2

# Dimensions derived from your H_punctured
n_qubits = H_punctured.shape[1] # Should be 11
k_checks = H_punctured.shape[0] # Should be 5
total_solver_qubits = 2 * n_qubits # 22

# Generate Errors
error_x = np.random.choice(
    [0, 1], size=(num_shots, n_qubits), p=[1 - rX * error_rate, rX * error_rate]
)
error_y = np.random.choice(
    [0, 1], size=(num_shots, n_qubits), p=[1 - rY * error_rate, rY * error_rate]
)
error_z = np.random.choice(
    [0, 1], size=(num_shots, n_qubits), p=[1 - rZ * error_rate, rZ * error_rate]
)

# Combine Y errors into X and Z components
error_x = (error_x + error_y) % 2
error_z = (error_z + error_y) % 2


In [57]:
H_np = np.array(H_punctured, dtype=int)

syndromes_x = (error_z @ H_np.T) % 2
syndromes_z = (error_x @ H_np.T) % 2
syndromes = np.hstack([syndromes_x, syndromes_z])


print(f"Running decoding for {num_shots} shots...\n")

for i in range(num_shots):
    syndrome_indices = np.nonzero(syndromes[i])[0].tolist()
    
    if not syndrome_indices:
        correction_vector = np.zeros(total_solver_qubits, dtype=int)
    else:
        hyperion.solve(SyndromePattern(syndrome_indices))
        subgraph_indices = hyperion.subgraph()
        correction_vector = np.zeros(total_solver_qubits, dtype=int)
        correction_vector[subgraph_indices] = 1

    correction_z = correction_vector[:n_qubits]
    correction_x = correction_vector[n_qubits:]

    residual_error_z = (correction_z + error_z[i]) % 2
    residual_error_x = (correction_x + error_x[i]) % 2

    total_residual_weight = np.sum(residual_error_x + residual_error_z)
    
    print(f"Shot {i}: Residual Weight = {total_residual_weight}")
    print(f"  Err Z: {error_z[i]} | Corr Z: {correction_z} | Res Z: {residual_error_z}")

Running decoding for 10 shots...

Shot 0: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 0] | Corr Z: [0 0 0 0 0 0 0 0 0 0 0] | Res Z: [0 0 0 0 0 0 0 0 0 0 0]
Shot 1: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 1] | Corr Z: [0 0 0 0 0 0 0 0 0 0 1] | Res Z: [0 0 0 0 0 0 0 0 0 0 0]
Shot 2: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 0] | Corr Z: [0 0 0 0 0 0 0 0 0 0 0] | Res Z: [0 0 0 0 0 0 0 0 0 0 0]
Shot 3: Residual Weight = 6
  Err Z: [0 1 1 0 0 0 0 0 0 0 0] | Corr Z: [1 0 0 0 0 0 0 0 0 0 0] | Res Z: [1 1 1 0 0 0 0 0 0 0 0]
Shot 4: Residual Weight = 10
  Err Z: [1 0 0 1 0 0 0 0 0 0 1] | Corr Z: [0 0 0 0 1 0 0 1 0 0 0] | Res Z: [1 0 0 1 1 0 0 1 0 0 1]
Shot 5: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 0] | Corr Z: [0 0 0 0 0 0 0 0 0 0 0] | Res Z: [0 0 0 0 0 0 0 0 0 0 0]
Shot 6: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 0] | Corr Z: [0 0 0 0 0 0 0 0 0 0 0] | Res Z: [0 0 0 0 0 0 0 0 0 0 0]
Shot 7: Residual Weight = 0
  Err Z: [0 0 0 0 0 0 0 0 0 0 0] | Corr Z